<a href="https://colab.research.google.com/github/robbarto2/GenAI-Foundations/blob/main/Instruction_Fine_Tuning_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install peft
!pip install torch
!pip install datasets

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset, DatasetDict

import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from google.colab import userdata
from huggingface_hub import login

# Retrieve the Hugging Face token securely
hf_token = userdata.get("HF_TOKEN")  # Replace 'HF_TOKEN' with your secret name

if not hf_token:
    raise ValueError("Hugging Face token not found in Colab secrets.")

# Log in using the token
login(token=hf_token)
print("Successfully logged into Hugging Face!")

In [ ]:
examples = []

In [ ]:
examples = [
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I haven't received my order #12345 yet. It's been a week.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Delayed
Reference: #12345
Action Taken: Order status check initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I received the wrong item in my order #67890.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Wrong Item Delivered
Reference: #67890
Action Taken: Replacement item initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #98765 is damaged. What should I do?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Damaged Item
Reference: #98765
Action Taken: Return and replacement process initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I want to cancel my order #54321.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Cancellation Request
Reference: #54321
Action Taken: Cancellation process initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I was charged twice for my order #11223. Please help.",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Billing Issue
Reference: #11223
Action Taken: Refund request initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "When will my order #44556 arrive?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: In Transit
Reference: #44556
Action Taken: Estimated delivery date provided

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #99988 was returned. When will I get my refund?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Returned
Reference: #99988
Action Taken: Refund process initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I can't track my order #66777. Can you help?",
        "output": """Thank you for contacting TechCorp Support.

Order Status: Tracking Issue
Reference: #66777
Action Taken: Tracking link resent

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
TechCorp Support"""
    },
        {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #56789 was supposed to arrive yesterday, but I haven't received it.",
        "output": """Thank you for contacting Tech Support.

Order Status: Delayed
Reference: #56789
Action Taken: Initiated a follow-up with the shipping carrier

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I received a damaged item in my order #98765. What should I do?",
        "output": """Thank you for contacting Tech Support.

Order Status: Damaged Item
Reference: #98765
Action Taken: Replacement process initiated. Return instructions sent.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I was charged twice for my order #11223. Please help.",
        "output": """Thank you for contacting Tech Support.

Order Status: Billing Issue
Reference: #11223
Action Taken: Refund process initiated for duplicate charge

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "When will my order #33445 arrive?",
        "output": """Thank you for contacting Tech Support.

Order Status: In Transit
Reference: #33445
Action Taken: Estimated delivery date provided via email

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I need to cancel my order #44556 immediately.",
        "output": """Thank you for contacting Tech Support.

Order Status: Cancellation Requested
Reference: #44556
Action Taken: Cancellation process initiated. Confirmation email sent.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I can't track my order #77889. The tracking link doesn't work.",
        "output": """Thank you for contacting Tech Support.

Order Status: Tracking Issue
Reference: #77889
Action Taken: Sent updated tracking link. Contacted carrier for more information.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "I returned my order #99000 last week. When will I get my refund?",
        "output": """Thank you for contacting Tech Support.

Order Status: Returned
Reference: #99000
Action Taken: Refund process initiated. Refund expected within 5 business days.

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    },
    {
        "instruction": "Respond to this customer inquiry following our format",
        "input": "My order #11122 arrived, but it's missing some items.",
        "output": """Thank you for contacting Tech Support.

Order Status: Incomplete Order
Reference: #11122
Action Taken: Missing items identified and replacement initiated

Need immediate assistance? Call: 1-800-TECH-HELP
Your satisfaction is our priority.

Best regards,
Tech Support"""
    }
]

In [ ]:
dataset = Dataset.from_list(examples)

In [ ]:
def format_example(example):
    instruction = example["instruction"]
    input_text = example["input"]
    output = example["output"]
    return {
        "text": f"### Instruction: {instruction}\n### Input: {input_text}\n### Response: {output}"
    }

formatted_examples = [format_example(ex) for ex in examples]
dataset = Dataset.from_list(formatted_examples)